In [16]:
# construct input for tool B and C from tool A's output
folder_name_A = '/Users/kavyasrinet/Desktop/other_actions/5/toolA/'
folder_name_B = '/Users/kavyasrinet/Desktop/other_actions/5/toolB/'
folder_name_C = '/Users/kavyasrinet/Desktop/other_actions/5/toolC/'
# folder_name_1 = '/Users/kavyasrinet/Downloads/'
# folder_name_2 = folder_name_1
tool_A_out_file = folder_name_A + 'all_agreements.txt'
tool_B_input_file = folder_name_B + 'input.txt'
tool_C_input_file = folder_name_C + 'input.txt'

from pprint import pprint
'''
command: Input.command

'''
import ast
import json
import copy

example_d = {}

def merge_indices(indices):
    a, b = indices[0]
    for i in range(1, len(indices)):
        x, y = indices[i]
        if x != b + 1:
            return indices
        else:
            b = y
    if a==b:
        return a
    return [a, b]


all_actions = {}
composites= []
with open(tool_A_out_file, "r") as f1, open(tool_B_input_file, 'w') as f2, open(tool_C_input_file, 'w') as f3:
    
    for line in f1.readlines():
        line  = line.strip()
        cmd, out = line.split("\t")
        words = cmd.split()
        
        words_copy = copy.deepcopy(words)
        action_dict = ast.literal_eval(out)
        action_type = action_dict['action_type'][1]
        
        all_actions[action_type] =  all_actions.get(action_type, 0)+ 1
        # write composite separately
        if action_type=='composite_action':
            composites.append(" ".join(words))
            continue
         
        # no need to annotate children of these two actions
        if action_type == 'noop':
            continue
            
        # find children that need to be re-annotated
        for key, val in action_dict.items():
            child_name = None
            # child needs annotation
            if val[0]== 'no':
                # insert "span"
                words_copy = copy.deepcopy(words)
                child_name = key
                
                write_line = ""
                write_line += " ".join(words) + "\t"
                #print(words, child_name, action_type, val[1])
                
                indices = merge_indices(val[1])
                span_text = None
                if type(indices) == list:
                    if type(indices[0]) == list:
                        # this means that indices were scattered and disjoint
                        for idx in indices:
                            words_copy[idx[0]] = "<span style='background-color: #FFFF00'>" + words_copy[idx[0]]
                            words_copy[idx[1]] = words_copy[idx[1]] + "</span>"
                    else:
                        words_copy[indices[0]] = "<span style='background-color: #FFFF00'>" + words_copy[indices[0]]
                        words_copy[indices[1]] = words_copy[indices[1]] + "</span>"
                else:
                    words_copy[indices] = "<span style='background-color: #FFFF00'>" + words_copy[indices] + "</span>"
                write_line += " ".join(words_copy) + "\t" + action_type + "\t" + child_name
                
                if child_name in ['reference_object', 
                                  'receiver_reference_object', 
                                  'source_reference_object']:
                    # write for tool C 
                    f3.write(write_line+ "\n")
                else:
                    # write for tool B
                    f2.write(write_line+ "\n")
                #print(write_line)
        
        #print("----"*5)



In [ ]:
# write out all unique composites
# only needs to be done once
mypath = '../../minecraft/python/craftassist/text_to_tree_tool/turk_data/tool1/humanbot/'
from os import walk
import ast

#all_composites = set()

f = []
for (dirpath, dirnames, filenames) in walk(mypath):
    if dirnames:
        #int(dirnames, dirpath)
        for dirname in dirnames:
            print("'" + dirname+"'")
            folder = str(dirpath) + dirname + "/" 
            file_name = folder + "all_agreements.txt"
            with open(file_name) as f:
                #print(file_name)
                for line in f.readlines():
                    line = line.strip()
                    text, d = line.split("\t")
                    action_type = ast.literal_eval(d)['action_type'][1]
                    if action_type == 'composite_action':
                        all_composites.add(text.strip())
print(len(all_composites))

In [ ]:
# now write composites to a file
with open('../../minecraft/python/craftassist/text_to_tree_tool/turk_data/all_composites.txt', 'w') as f:
    for c in all_composites:
        f.write(c + "\n")

In [11]:
# construct input for tool D from tool C's output
# inpout for tool D is : 
# text \t highlighted_text \t "reference_object" \t "comparison"
# folder_name_C = '../../minecraft/python/craftassist/text_to_tree_tool/turk_data/new_dance_form_data/next_20/tool3/'
# folder_name_D = '../../minecraft/python/craftassist/text_to_tree_tool/turk_data/new_dance_form_data/next_20/tool4/'
folder_name_C = '/Users/kavyasrinet/Desktop/other_actions/0/toolC/'
folder_name_D = '/Users/kavyasrinet/Desktop/other_actions/0/toolD/'
tool_C_out_file = folder_name_C + 'all_agreements.txt'
tool_D_input_file = folder_name_D + 'input.txt'

from pprint import pprint
'''
command: Input.command

'''
import copy
example_d = {}

def merge_indices(indices):
    a, b = indices[0]
    for i in range(1, len(indices)):
        x, y = indices[i]
        if x != b + 1:
            return indices
            # raise NotImplementedError("Spans not continuous during merging!!!")
        else:
            b = y
    if a==b:
        return a
    return [a, b]
import ast
import json


all_actions = {}
with open(tool_C_out_file, "r") as f1, open(tool_D_input_file, 'w') as f2:
    composites= []
    
    for line in f1.readlines():
        line  = line.strip()
        cmd, action_child_name , out = line.split("\t")
        words = cmd.split()
        
        words_copy = copy.deepcopy(words)
        #pprint(out)
        if action_child_name not in ['reference_object', 
                                     'receiver_reference_object',
                                     'source_reference_object']:
            print("Not a reference object!!!")
            continue
        out_dict = json.loads(out)
        ref_obj_dict = out_dict[action_child_name]
#         print(out_dict)
#         print(ref_obj_dict)
#         break
            
        # find children that need to be re-annotated
        for key, val in ref_obj_dict.items():
            child_name = None
            # child needs annotation
            if val[0]== 'no':
                # insert "span"
                words_copy = copy.deepcopy(words)
                child_name = key
                
                write_line = ""
                write_line += " ".join(words) + "\t"
                #print(words, child_name, action_type, val[1])
                
                indices = merge_indices(val[1])
                span_text = None
                if type(indices) == list:
                    if type(indices[0]) == list:
                        # this means that indices were scattered and disjoint
                        for idx in indices:
                            words_copy[idx[0]] = "<span style='background-color: #FFFF00'>" + words_copy[idx[0]]
                            words_copy[idx[1]] = words_copy[idx[1]] + "</span>"
                    else:
                        words_copy[indices[0]] = "<span style='background-color: #FFFF00'>" + words_copy[indices[0]]
                        words_copy[indices[1]] = words_copy[indices[1]] + "</span>"
                else:
                    words_copy[indices] = "<span style='background-color: #FFFF00'>" + words_copy[indices] + "</span>"
                write_line += " ".join(words_copy) + "\t" + action_child_name + "\t" + child_name
                f2.write(write_line+ "\n")
